In [1]:
import pandas as pd
import os
import sys
import json

from tqdm.notebook import tqdm

# from matplotlib import pyplot as plt

In [2]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [3]:
from helper import calculate_len_and_time,sum_cols

In [4]:
pd.set_option('max_colwidth',None)

In [5]:
def fun(teamid,teams):
    if teamid == teams[0]:
        return teams[1]
    else:
        return teams[0]


In [52]:
ex_list = ['FormationChange','SubstitutionOn','SubstitutionOff','Card']

In [53]:
df_summary = pd.DataFrame(None,columns=['match_id','index','possession_team','minute','second','player','x','y','endX','endY',
                                            'end_action','end_team','eventendX','eventendY','duration','length','opposition_team','event_type'])

# read the file
for match_id in tqdm(range(0,380)):
    with open(f'../data/seq/{match_id}.json') as f:
        t_seq = json.load(f)

        length = []
        for i in t_seq:
            length.append(i[1]-i[0])

        df_all = pd.read_csv(f"../data/20_21/{match_id}.csv")
        
        df_all = df_all.query('type_displayName not in @ex_list').reset_index()  

        st_time = [x[0]*60+x[1] for x in df_all.iloc[[x[0] for x in t_seq]][['minute','second']].values]
        end_time = [x[0]*60+x[1] for x in df_all.iloc[[x[1] for x in t_seq]][['minute','second']].values]

        duration = []
        for e,s in zip(end_time,st_time):
            duration.append(e-s)

        df = pd.DataFrame(None)

        df['match_id'] = [match_id]*len(t_seq)
        df['index'] = [x[0] for x in t_seq]
        df[['possession_team','minute','second','player','x','y','endX','endY']] = df_all.iloc[[x[0] for x in t_seq]][['teamId','minute','second','name','x','y','endX','endY']].values

        teams = df_all.teamId.unique()

        df[['end_action','end_team','eventendX','eventendY']] = df_all.iloc[[x[1] for x in t_seq]][['type_displayName','teamId','x','y']].values

        df['duration'] = duration

        df['length'] = length

        df['opposition_team'] = list(map(lambda teamid:fun(teamid,teams),df_all.iloc[[x[0] for x in t_seq]]['teamId']))

        df['event_type'] = df_all.iloc[[x[1] for x in t_seq]]['satisfiedEventsTypes'].values
        
        df_summary = df_summary.append(df)

  0%|          | 0/380 [00:00<?, ?it/s]

In [54]:
df_summary

,match_id,index,possession_team,minute,second,player,x,y,endX,endY,end_action,end_team,eventendX,eventendY,duration,length,opposition_team,event_type
0,0,15,184,1,6.0,Erik Pieters,60.0,100.0,78.0,93.5,Pass,32,40.2,65.1,13.0,7,32,"[90, 116, 29, 34, 36, 214, 217]"
1,0,31,32,1,47.0,Bruno Fernandes,60.4,0.0,41.2,5.5,Foul,32,40.6,9.2,19.0,6,184,"[90, 62]"
2,0,84,32,3,59.0,Aaron Wan-Bissaka,63.7,0.0,66.0,9.2,BallRecovery,184,36.4,73.4,5.0,3,184,[92]
3,0,152,184,7,24.0,Erik Pieters,38.1,100.0,37.5,89.1,Foul,32,57.8,10.6,19.0,10,32,"[90, 62]"
4,0,194,184,10,17.0,Erik Pieters,67.5,100.0,81.5,85.1,Pass,32,32.1,13.2,16.0,11,32,"[90, 116, 29, 34, 36, 214, 217]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,379,1473,161,88,23.0,Nélson Semedo,43.4,0.0,27.7,6.4,BallTouch,32,41.5,83.6,25.0,14,32,[90]
33,379,1488,161,88,56.0,Romain Saïss,40.0,0.0,32.9,26.0,Clearance,32,15.7,5.6,32.0,14,32,"[90, 93, 94, 214]"
34,379,1503,161,89,38.0,Marçal,77.9,100.0,70.7,91.7,Clearance,32,6.8,44.2,29.0,12,32,"[90, 93, 94, 56, 214]"
35,379,1520,32,90,41.0,Brandon Williams,19.2,0.0,26.9,5.3,Pass,161,66.7,95.3,1.0,1,161,"[90, 118, 116, 29, 138, 204, 35, 37, 216, 217]"


In [55]:
df_summary.loc[df_summary['event_type'].str.contains("211"),'end_action'] = "Throw-in"

In [56]:
df_summary.loc[df_summary['event_type'].str.contains("211")]

,match_id,index,possession_team,minute,second,player,x,y,endX,endY,end_action,end_team,eventendX,eventendY,duration,length,opposition_team,event_type
10,8,292,211,16,36.0,Tariq Lamptey,30.2,0.0,65.7,0.0,Throw-in,211,30.2,0.0,0.0,0,15,"[90, 211, 35, 215]"
11,18,261,18,18,15.0,Ryan Bertrand,40.4,100.0,73.1,100.0,Throw-in,18,40.4,100.0,0.0,0,30,"[90, 211, 35, 216]"
12,45,800,13,42,7.0,Kieran Tierney,14.5,100.0,44.9,100.0,Throw-in,13,14.5,100.0,0.0,0,167,"[90, 211, 35, 215]"
26,92,1022,13,61,49.0,Kieran Tierney,38.9,100.0,71.4,100.0,Throw-in,13,38.9,100.0,0.0,0,184,"[90, 211, 35, 216]"
10,130,379,211,24,35.0,Joël Veltman,75.0,0.0,90.0,0.0,Throw-in,211,75.0,0.0,0.0,0,163,"[90, 211, 35, 216]"
36,131,1184,184,82,57.0,Charlie Taylor,5.6,100.0,17.7,100.0,Throw-in,184,5.6,100.0,0.0,0,161,"[90, 211, 35, 214]"
0,143,96,163,4,22.0,Jayden Bogle,26.4,100.0,55.3,100.0,Throw-in,163,26.4,100.0,0.0,0,167,"[90, 211, 35, 215]"
28,144,1022,18,55,0.0,Jan Bednarek,53.5,0.0,79.1,0.0,Throw-in,18,53.5,0.0,0.0,0,15,"[90, 211, 35, 216]"
35,180,1255,175,84,8.0,Darnell Furlong,22.0,0.0,58.6,0.0,Throw-in,175,22.0,0.0,0.0,0,211,"[90, 211, 35, 215]"
29,207,940,175,55,2.0,Darnell Furlong,14.6,0.0,35.1,0.0,Throw-in,175,14.6,0.0,0.0,0,23,"[90, 211, 35, 215]"


In [57]:
df_summary.end_action.unique()

array(['Pass', 'Foul', 'BallRecovery', 'CornerAwarded', 'Interception',
       'SavedShot', 'End', 'BallTouch', 'Tackle', 'KeeperPickup',
       'BlockedPass', 'OffsideGiven', 'MissedShots', 'Claim', 'Clearance',
       'Goal', 'TakeOn', 'OffsidePass', 'ShieldBallOpp', 'Throw-in',
       'KeeperSweeper', 'Smother', 'Punch', 'OffsideProvoked', 'Aerial',
       'Save', 'GoodSkill'], dtype=object)

In [61]:
df_summary.reset_index(drop=True,inplace=True)

In [62]:
df_summary

,match_id,index,possession_team,minute,second,player,x,y,endX,endY,end_action,end_team,eventendX,eventendY,duration,length,opposition_team,event_type
0,0,15,184,1,6.0,Erik Pieters,60.0,100.0,78.0,93.5,Pass,32,40.2,65.1,13.0,7,32,"[90, 116, 29, 34, 36, 214, 217]"
1,0,31,32,1,47.0,Bruno Fernandes,60.4,0.0,41.2,5.5,Foul,32,40.6,9.2,19.0,6,184,"[90, 62]"
2,0,84,32,3,59.0,Aaron Wan-Bissaka,63.7,0.0,66.0,9.2,BallRecovery,184,36.4,73.4,5.0,3,184,[92]
3,0,152,184,7,24.0,Erik Pieters,38.1,100.0,37.5,89.1,Foul,32,57.8,10.6,19.0,10,32,"[90, 62]"
4,0,194,184,10,17.0,Erik Pieters,67.5,100.0,81.5,85.1,Pass,32,32.1,13.2,16.0,11,32,"[90, 116, 29, 34, 36, 214, 217]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14886,379,1473,161,88,23.0,Nélson Semedo,43.4,0.0,27.7,6.4,BallTouch,32,41.5,83.6,25.0,14,32,[90]
14887,379,1488,161,88,56.0,Romain Saïss,40.0,0.0,32.9,26.0,Clearance,32,15.7,5.6,32.0,14,32,"[90, 93, 94, 214]"
14888,379,1503,161,89,38.0,Marçal,77.9,100.0,70.7,91.7,Clearance,32,6.8,44.2,29.0,12,32,"[90, 93, 94, 56, 214]"
14889,379,1520,32,90,41.0,Brandon Williams,19.2,0.0,26.9,5.3,Pass,161,66.7,95.3,1.0,1,161,"[90, 118, 116, 29, 138, 204, 35, 37, 216, 217]"


In [63]:
df_summary.to_csv("../data/throw_in.csv",index=False)

In [70]:
df_summary.groupby(by = 'possession_team',sort=False).median()

,duration
possession_team,
184,11.0
32,15.0
24,10.0
167,19.0
18,9.0
162,8.0
13,15.0
170,11.0
19,11.0


In [71]:
df_summary.groupby(by = 'opposition_team',sort=False).median()

,duration
opposition_team,
32,10.0
184,12.0
167,11.0
24,10.0
162,12.0
18,12.0
170,12.0
13,14.0
26,9.0


In [98]:
gk = df_summary.groupby(['possession_team','end_action'])

In [108]:
gk.size().groupby(level=1).max()

end_action
Aerial               2
BallRecovery       135
BallTouch           71
BlockedPass         44
Claim               11
Clearance           63
CornerAwarded       44
End                 11
Foul               123
Goal                12
GoodSkill            1
Interception        60
KeeperPickup        58
KeeperSweeper        2
MissedShots         37
OffsideGiven        19
OffsidePass         15
OffsideProvoked      7
Pass               171
Punch                2
Save                 1
SavedShot           56
ShieldBallOpp        6
Smother              1
Tackle              70
TakeOn              11
Throw-in             3
dtype: int64